In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    # RIP Training time
    raise SystemError('GPU device not found')
    print('Found GPU at: {}'.format(device_name))

In [2]:
TRAIN_DATA_ZIP_PATH = "vn_handwritten_images.zip"

In [3]:
import zipfile
with zipfile.ZipFile(TRAIN_DATA_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall("vietnamese_hcr/raw")

In [4]:
import pathlib

current_path = pathlib.Path('./vietnamese_hcr/').absolute()

current_path

WindowsPath('d:/code-ml/DATN_Vietnamese_OCR/vietnamese_hcr')

In [5]:
import os

DATA_PATH = os.path.join(str(current_path), "data")
TEST_FOLDER = os.path.join(DATA_PATH, "test")
TRAIN_FOLDER = os.path.join(DATA_PATH, "train")

# our raw folders
RAW_FOLDER = os.path.join(str(current_path),"raw")

# label json files
TRAIN_JSON = os.path.join(RAW_FOLDER, "labels.json")

In [6]:
RAW_FOLDER

'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw'

In [7]:
TRAIN_JSON

'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\labels.json'

In [8]:
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
if not os.path.exists(TEST_FOLDER):
    os.makedirs(TEST_FOLDER)
if not os.path.exists(TRAIN_FOLDER):
    os.makedirs(TRAIN_FOLDER)

In [9]:
import json
# open these label jsons files
with open(TRAIN_JSON, 'r', encoding='utf8') as file:
    train_labels = json.load(file)

In [10]:
train_labels

{'1.jpg': 'Số 3 Nguyễn Ngọc Vũ, Hà Nội',
 '2.jpg': 'Số 30 Nguyên Hồng, Láng Hạ, Đống Đa, Hà Nội',
 '3.jpg': '58 Thái Thịnh, Đống Đa, Hà Nội',
 '4.jpeg': 'Số 370/8 khu phố 5B, phường Tân Biên, Biên Hòa, Đồng Nai',
 '5.jpg': 'Vĩnh Trung Plaza, B, 255-257 đường Hùng Vương, phường Vĩnh Trung',
 '6.jpg': 'Tòa nhà 34T, Hoàng Đạo Thúy, Hà Nội',
 '7.jpg': '40 Cát Linh, Đống Đa, Hà Nội',
 '8.jpg': 'phòng 101, tầng 1, lô 04-TT5B, khu đô thị Tây Nam Linh Đàm',
 '9.JPG': 'Nhà 87 ngõ 416 Đê La Thành',
 '10.JPG': 'Up coworking Space, 89 Láng Hạ, Hà Nội',
 '11.jpg': '192 Ngô Đức Kế, quận 1, Hồ Chí Minh',
 '12.jpg': 'số 5 Công Trường Mê Linh, phường Bến Nghé, quận 1',
 '13.jpg': '90A đường Mai Xuân Thưởng, tỉnh Gia Lai',
 '14.jpg': '96/7/12B Phạm Văn Đồng, thành phố Pleiku',
 '15.jpg': '168 Ngô Gia Tự, thành phố Hà Tĩnh',
 '0000_samples.png': 'Số 253 đường Trần Phú, Thị trấn Nam Sách, Huyện Nam Sách, Hải Dương',
 '0001_samples.png': 'Số 289 Đinh Bộ Lĩnh, Phường 26, Quận Bình Thạnh, TP Hồ Chí Minh',
 '

In [11]:
# find all characters in labels (so we don't blindly add chacters not even existed in our dataset)
char_list= set()
for label in train_labels.values():
    char_list.update(set(label))
char_list=sorted(char_list)

In [12]:
print(f'There are {len(char_list)} unique characters in this dataset.')

# show all possible labels characters
print("".join(char_list))


There are 140 unique characters in this dataset.
 #'()+,-./0123456789:ABCDEFGHIJKLMNOPQRSTUVWXYabcdeghiklmnopqrstuvwxyzÂÊÔàáâãèéêìíòóôõùúýăĐđĩũƠơưạảấầẩậắằẵặẻẽếềểễệỉịọỏốồổỗộớờởỡợụủỨứừửữựỳỵỷỹ


In [13]:
# convert the words to array of indexs based on the char_list
def encode_to_labels(txt):
    # encoding each output word into digits of indexes
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print("No found in char_list :", char)
        
    return dig_lst

In [14]:
# example
encode_to_labels("hello world")

[52, 50, 55, 55, 58, 0, 66, 58, 61, 55, 49]

In [15]:
# find all image paths for train folder 1 and 2 and test folder
train_image_path = []

for item in pathlib.Path(RAW_FOLDER).glob('**/*'):
    if item.is_file() and item.suffix not in [".json"]:
        train_image_path.append(str(item))

In [16]:
train_image_path[:20]

['d:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0000_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0001_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0002_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0003_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0004_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0005_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0006_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0007_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0008_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0009_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0010_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0011_samples.png',
 'd:\\code-ml\\D

Preprocessing

- Find all widths and heights of images
- Use openCV to read image
- Preprocess images (like converting images to greyscale)
- Resize images so all images will have the same size
- Split your dataset into trainset and testset
- Build CRNN model with CTC loss
- Prediction
- Calculate metrics for SER, WER and CER

In [17]:
# Matching full absoluate paths and labels instead of filenames and labels 
dict_filepath_label={}
raw_data_path = pathlib.Path(os.path.join(RAW_FOLDER))
for item in raw_data_path.glob('**/*.*'):
    file_name=str(os.path.basename(item))
    if file_name != "labels.json":
      label = train_labels[file_name]
      dict_filepath_label[str(item)]=label

In [18]:
dict_filepath_label

{'d:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0000_samples.png': 'Số 253 đường Trần Phú, Thị trấn Nam Sách, Huyện Nam Sách, Hải Dương',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0001_samples.png': 'Số 289 Đinh Bộ Lĩnh, Phường 26, Quận Bình Thạnh, TP Hồ Chí Minh',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0002_samples.png': 'Số 246E/2, Khu phố 1B, Phường An Phú, Tx Thuận An, Bình Dương',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0003_samples.png': '42 Đặng Thị Nhu, Phường Nguyễn Thái Bình, Quận 1, TP Hồ Chí Minh',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0004_samples.png': '200 Phan Bội Châu, Phường Trường An, Thành phố Huế, Thừa Thiên - Huế',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0005_samples.png': '27A Hoàng Việt, Phường 4, Quận Tân Bình, TP Hồ Chí Minh',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0006_samples.png': 'Số 113 Đường Hưng Thái 

In [19]:
# find the maximum label length
label_lens= []
for label in dict_filepath_label.values():
    label_lens.append(len(label))
max_label_len = max(label_lens)

print(f'max label length: {max_label_len}')

max label length: 69


In [20]:
all_image_paths = list(dict_filepath_label.keys())

all_image_paths[:10]

['d:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0000_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0001_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0002_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0003_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0004_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0005_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0006_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0007_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0008_samples.png',
 'd:\\code-ml\\DATN_Vietnamese_OCR\\vietnamese_hcr\\raw\\data\\0009_samples.png']

In [21]:
import cv2

widths = []
heights = []
for img_path in all_image_paths:
    img = cv2.imread(img_path)
    (h, w, _) = img.shape
    heights.append(h)
    widths.append(w)

In [22]:
min_height = min(heights)
max_height = max(heights)
min_width = min(widths)
max_width = max(widths)

# this information is useful for making decision for padding and resizing
(min_height, max_height, min_width, max_width)

(94, 376, 955, 2694)

In [23]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2

train_image_paths, val_image_paths = train_test_split(all_image_paths, test_size=TEST_SIZE, random_state=42)

In [24]:
# Constants (we choose this number as we later discover that the output of our CNN is (1,9,512) from out input)
TIME_STEPS = 240

BUILD THE PIPELINE FOR THE IMAGES, definitely this is super basic pipeline